<h1>Analysis</h1>
<h2></h2>

</p>It is pretty clear from the data that there is a strong correlation between tempature and latitue.
    as expected there are higher tempatures the closer to the equator and lower temperatures the farther 
    away we get from the equator in either direction. Although there is a sharper decline the further south we go 
    and not as much in the northern hemisphere the further north we go</p> 
<img src="Temp_plot.png" alt="Temperature Plot">

<h2></h2>
<p>According to the data, latitude plays almost no part in how cloudy it is. There seems to be cloudiness all over the world 
   no matter what latitude you are at.</p>
<img src="Cloudiness_plot.png" alt="Cloudiness Plot">

<h2></h2> 
<p>Humidity and latitude have a pretty weak correlation. Humidity around the world seems to be pretty high. Though around the 
  equator seems to be the least humid places dispite being a higher temperature then everywhere else in the world</p>
<img src="Humidity_plot.png" alt="Humidty Plot">
  

In [ ]:
# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
from pprint import pprint
from datetime import datetime

# Import API key
from api_keys import weather_api_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [ ]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

## Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'being processed (with the city number and city name).


In [ ]:
query_url = "http://api.openweathermap.org/data/2.5/weather?appid=" + weather_api_key + "&units=imperial" + "&q="

tempList = []
humList = [] 
cloudList = []
latList = []
longList = []
windList = []
countryList = []
cityList = []
dateList = []


for city in cities:
    response = requests.get(query_url + city).json()
    
    try:
        print(f'Gathering Data for {city} with ID {response["id"]}')
        tempList.append(response["main"]["temp_max"])
        humList.append(response["main"]["humidity"])
        cloudList.append(response["clouds"]["all"])
        latList.append(response["coord"]["lat"])
        longList.append(response["coord"]["lon"])
        windList.append(response["wind"]["speed"])
        countryList.append(response["sys"]["country"])
        cityList.append(response["name"])
        dateList.append(response["dt"])
                           
    except KeyError as e:
        print(f"City Not Found: {city}")
    pass



### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [ ]:
weather_dict = {"City": cityList, 
                "Country": countryList, 
                "Temp": tempList, 
                "Humidity": humList, 
                "Lat": latList, 
                "Long": longList, 
                "Cloud": cloudList, 
                "Wind": windList, 
                "Date": dateList
                }
weather_df = pd.DataFrame(weather_dict)
weather_df.to_csv("Output/weather_data.csv")
weather_df.head(500)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.
badHumidity = weather_df.loc[weather_df["Humidity"] > 100 ]
badHumidity


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


In [ ]:
# Extract relevant fields from the data frame


# Export the City_Data into a csv


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

In [ ]:
plt.scatter(weather_df["Lat"], weather_df["Temp"])
plt.xlabel("Latitude")
plt.ylabel("Temperature (F)")
plt.title(f'Latitude vs Temperature {datetime.now().strftime("%Y-%m-%d")}')
plt.savefig("Temp_plot.png")
plt.show()


## Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather_df["Lat"], weather_df["Humidity"])
plt.xlabel("Latitude")
plt.ylabel("Humidity")
plt.title(f'Latitude vs Humidity {datetime.now().strftime("%Y-%m-%d")}')
plt.savefig("Humidity_plot.png")
plt.show()

## Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather_df["Lat"], weather_df["Cloud"])
plt.xlabel("Latitude")
plt.ylabel("Cloudiness")
plt.title(f'Latitude vs Cloudiness {datetime.now().strftime("%Y-%m-%d")}')
plt.savefig("Cloudiness_plot.png")
plt.show()

## Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather_df["Lat"], weather_df["Wind"])
plt.xlabel("Latitude")
plt.ylabel("Wind Speed")
plt.title(f'Latitude vs Wind Speed {datetime.now().strftime("%Y-%m-%d")}')
plt.savefig("Wind_Plot.png")
plt.show()

## Linear Regression

In [ ]:
# OPTIONAL: Create a function to create Linear Regression plots
def plotLinRegress(val1, lat, xlabel, ylabel,title): 
    (slope, intercept, rvalue, pvalue, stderr) = linregress(lat,val1)
    regress_values = val1 * slope + intercept
    line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
    plt.scatter(lat, val1)
    plt.plot(val1, regress_values, "r-")
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.show()
    

    

In [ ]:
# Create Northern and Southern Hemisphere DataFrames
northernhem_df = weather_df.loc[weather_df["Lat"] > 0]
southernhem_df = weather_df.loc[weather_df["Lat"] < 0]
#northernhem_df
southernhem_df

##  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
xlabel = "Latitude"
ylabel = "Max Temp"
title = "Northern Hemisphere - Max Temp vs Latitude Linear Regression"
MaxTemp = northernhem_df["Temp"]
Lat = northernhem_df["Lat"]
plt.savefig("NH_max_regress.png")

plotLinRegress(MaxTemp,Lat,xlabel,ylabel, title )


####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
title = "Southern Hemisphere - Max Temp vs Latitude Linear Regression"
MaxTemp = southernhem_df["Temp"]
Lat = southernhem_df["Lat"]
plt.savefig("SH_max_regress.png")


plotLinRegress(MaxTemp,Lat,xlabel,ylabel, title )


####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Humidity (%)'
title = "Northern Hemisphere - Humidity (%) vs Latitue Linear Regression"
nHumidity = northernhem_df["Humidity"]
Lat = northernhem_df["Lat"]
plt.savefig("NH_hum_regress.png")


plotLinRegress(nHumidity,Lat,xlabel,ylabel, title )

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Humidity (%)'
title = "Southern Hemisphere - Humidity (%) vs Latitue Linear Regression"
Humidity = southernhem_df["Humidity"]
Lat = southernhem_df["Lat"]
plt.savefig("SH_hum_regress.png")


plotLinRegress(Humidity,Lat,xlabel,ylabel, title )

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Cloudiness (%)'
title = "Northern Hemisphere - Cloudiness (%) vs Latitue Linear Regression"
Humidity = northernhem_df["Humidity"]
Lat = northernhem_df["Lat"]
plt.savefig("NH_cloud_regress.png")


plotLinRegress(Humidity,Lat,xlabel,ylabel, title )

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Cloudiness (%)'

title = "Southern Hemisphere - Humidity (%) vs Latitue Linear Regression"
Humidity = southernhem_df["Humidity"]
Lat = southernhem_df["Lat"]
plt.savefig("SH_cloud_regress.png")


plotLinRegress(Humidity,Lat,xlabel,ylabel, title )

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Wind (mph)'
xlabel = "Latitude"
title = "Northern Hemisphere - Wind Speed (mph) vs Latitude Linear Regression"
Wind = northernhem_df["Wind"]
Lat = northernhem_df["Lat"]
plt.savefig("NH_wind_regress.png")


plotLinRegress(Wind,Lat,xlabel,ylabel, title )

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
ylabel = 'Wind (mph)'

title = "Southern Hemisphere - Wind Speed (mph) vs Latitue Linear Regression"
Wind = southernhem_df["Wind"]
Lat = southernhem_df["Lat"]
plt.savefig("NH_wind_regress.png")

plotLinRegress(Wind,Lat,xlabel,ylabel, title )